In [1]:
import netCDF4 as nc
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
%matplotlib inline

In [2]:
HSY = xr.open_dataset('HSY_daily_cftime.nc')
arr = HSY.HotSpot
arr_round = xr.where(arr < 0.00005, 0, arr)
arr_int = xr.where(arr < 0.00005 , 0, 1)

In [3]:
def one(x):
    return xr.where((x.rolling(time=3, center = True).sum().fillna(1)) == 0, 0, 1)
arr_one = arr_int.groupby('time.year').map(one)

In [ ]:
def rolling_mean(x):
    return x.rolling(time=3, center=True).mean()
daily = arr_round.groupby('time.year').map(rolling_mean)
daily.to_netcdf('daily_00005_.nc')

In [4]:
daily = (xr.open_dataset('daily_00005_.nc')).HotSpot

In [5]:
HSpeak = daily.groupby('time.year').max()

In [14]:
HSpeak.to_netcdf('HSpeak.nc')

In [6]:
def where_max(x):
    maxi = x.max(dim = 'time')
    return xr.where(x == maxi, 1, 0)

In [7]:
max_mask = daily.groupby('time.year').map(where_max)

In [8]:
def max_loc(x):
    return x.argmax(dim = 'time')

In [9]:
max_mask_year = max_mask.groupby('time.year').map(max_loc)

In [12]:
max_mask_year.to_netcdf('max_mask_year.nc')

In [ ]:
import scipy as sc
import scipy.ndimage

In [ ]:
def a_label(x): 
    label = (scipy.ndimage.label(x)[0])
    return np.array(label)

In [ ]:
def stacked(x): 
    stack = x.stack(gridcell=['lat', 'lon'])
    return (stack.groupby("gridcell").map(a_label).unstack("gridcell"))

In [ ]:
continuous_period = arr_one.groupby('time.year').map(stacked)

In [ ]:
continuous_period.to_netcdf('continous_period_august_4.nc')

In [10]:
continuous_period = xr.open_dataset('continous_period_august_4.nc')

In [11]:
continuous_period = continuous_period.HotSpot

In [12]:
max_group = continuous_period.where((max_mask == 1), 0)

In [13]:
query_daily = continuous_period.where((continuous_period > 0) & (continuous_period == max_group))

In [14]:
def fill(x):
    forward = x.ffill(dim = 'time')
    backward = forward.bfill(dim = 'time')
    return backward

In [15]:
filled = query_daily.groupby('time.year').map(fill)

In [16]:
filled.to_netcdf('filled_august_4.nc')

In [20]:
filled = xr.open_dataset('filled_august_4.nc')

In [21]:
filled = filled.HotSpot

In [18]:
query_continuous = xr.where(continuous_period == filled, continuous_period, 0)

In [23]:
query_continuous.to_netcdf('query_00005_.nc')

In [5]:
def d_acc(acc):
    return acc.cumsum(dim='time')-acc.cumsum(dim='time').where(acc.values == 0).ffill(dim='time').fillna(0)

In [6]:
cumsum = query_continuous.groupby('time.year').map(d_acc)

In [7]:
cumsum.to_netcdf('cumsum_00005_.nc')

In [20]:
cumsum = (xr.open_dataset('cumsum_00005_.nc')).HotSpot

In [21]:
def first(x):
    return xr.where(x>0, x, np.inf).argmin(dim = 'time')

def last(x):
    return xr.where(x>0, x, 0).argmax(dim = 'time')

In [22]:
last = cumsum.groupby('time.year').map(last)
first = cumsum.groupby('time.year').map(first)

In [30]:
last.to_netcdf('last_HS_c_raw.nc')
first.to_netcdf('first_HS_C_raw.nc')

In [ ]:
first_plus1 = xr.where(first==0, first+1, first)
last_minus1 = xr.where(last==364, last-1, last)

In [ ]:
first_plus1.to_netcdf('firstHS_c.nc')
last_minus1.to_netcdf('lastHS_c.nc')

In [ ]:
first_plus1 = xr.open_dataset('firstHS_c.nc')
last_minus1 = xr.open_dataset('lastHS_c.nc')

In [ ]:
first_plus1 = first_plus1.HotSpot
last_minus1 = last_minus1.HotSpot

In [ ]:
Dc_pre = (last_minus1 - first_plus1)

In [ ]:
Dc = xr.where(Dc_pre>0, Dc_pre+1, 0)

In [ ]:
def f(x):
    return np.count_nonzero(x, axis=-1)

g = arr_round.groupby('time.year')
mask = xr.apply_ufunc(f, g, input_core_dims = [['time']])

In [ ]:
Dc_mask = Dc.where(mask > 3, 0)

In [ ]:
year_mask = daily.groupby('time.year').max()

In [ ]:
Dc_mask13 = Dc_mask.where(year_mask >= 0, np.nan)

In [ ]:
Dc_mask13.to_netcdf('Dc_00005_.nc')

In [ ]:
Dc = xr.open_dataset('DC_00005_.nc')

In [ ]:
Dc = Dc.HotSpot

In [ ]:
def positive_three(x):
    return xr.where((x.rolling(time=3, center = True).sum().fillna(0)) == 3, 1, 0)
arr_one = arr_int.groupby('time.year').map(positive_three)

def back_fill(x):
    make_nan = xr.where(x == 0, np.nan, x)
    backward = make_nan.bfill(dim = 'time', limit = 2)
    forward = backward.ffill(dim = 'time', limit = 2)
    return forward.fillna(0)
arr_one_back = arr_one.groupby('time.year').map(back_fill)

#def first(x):
#    return xr.where(x>0, x, np.inf).argmin(dim = 'time')
#first = cumsum.groupby('time.year').map(first)
#first_plus1 = xr.where(first==0, first+1, first)

def d_first(x):
    acc = x.cumsum(dim = 'time')
    return xr.where(acc>0, acc, np.inf).argmin(dim = 'time')
first_HS = arr_one_back.groupby('time.year').map(d_first)
first_HS_plus1 = xr.where(first_HS==0, first_HS+1, first_HS)
first_HS_end = xr.where(first_HS_plus1 >= 361, 0, first_HS_plus1)

def d_last(x):
    acc = x.cumsum(dim = 'time')
    return acc.argmax(dim = 'time')
last_HS = arr_int.groupby('time.year').map(d_last)
last_HS_minus1 = xr.where(last_HS==361, last_HS-1, last_HS)
last_HS_start = xr.where(last_HS_minus1 <= 3, 0, last_HS_minus1)

HS_last = xr.where(first_HS_end == 0, 0, last_HS_start)
HS_begin = xr.where(last_HS_start == 0, 0, last_HS_start)

In [ ]:
Dp_pre = (first_plus1 - first_HS_plus1) 
Dp = xr.where(Dc <= 4, 0, Dp_pre)
dp_plus = xr.where(Dp>0, Dp+1, 0)

In [ ]:
Dp_mask = dp_plus.where(mask > 3, 0)

In [ ]:
Dp_mask13 = Dp_mask.where(year_mask >= 0, np.nan)

In [ ]:
Dp_mask13.to_netcdf('Dp_00005_.nc')